<a href="https://colab.research.google.com/github/RiyaMaurya1/Deep_learning/blob/main/Resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os

In [ ]:
os.environ['KAGGLE_USERNAME'] = 'riyaasat'
os.environ['KAGGLE_KEY'] = "551f8333d17ea1d826c72c8ad68f1ed4"

In [ ]:
!kaggle datasets download utkarshsaxenadn/fast-food-classification-dataset

 99% 812M/821M [00:11<00:00, 72.8MB/s]
100% 821M/821M [00:11<00:00, 75.3MB/s]


In [ ]:
from IPython.display import clear_output
!unzip fast-food-classification-dataset.zip

clear_output()

Streaming output truncated to the last 5000 lines.
  inflating: Fast Food Classification V2/TFRecords/Train/Taquito_14059.tfrecord  
  inflating: Fast Food Classification V2/TFRecords/Train/Taquito_14060.tfrecord  
  inflating: Fast Food Classification V2/TFRecords/Train/Taquito_14061.tfrecord  
  inflating: Fast Food Classification V2/TFRecords/Train/Taquito_14062.tfrecord  
  inflating: Fast Food Classification V2/TFRecords/Train/Taquito_14063.tfrecord  
  inflating: Fast Food Classification V2/TFRecords/Train/Taquito_14064.tfrecord  
  inflating: Fast Food Classification V2/TFRecords/Train/Taquito_14065.tfrecord  
  inflating: Fast Food Classification V2/TFRecords/Train/Taquito_14066.tfrecord  
  inflating: Fast Food Classification V2/TFRecords/Train/Taquito_14067.tfrecord  
  inflating: Fast Food Classification V2/TFRecords/Train/Taquito_14068.tfrecord  
  inflating: Fast Food Classification V2/TFRecords/Train/Taquito_14069.tfrecord  
  inflating: Fast Food Classification V2/TFReco

In [ ]:
%load_ext tensorboard

In [ ]:
train_dir = "Fast Food Classification V2/Train"
test_dir = "Fast Food Classification V2/Test"
valid_dir = "Fast Food Classification V2/Valid"

In [ ]:
import cv2
# to see image without any show method along with wait.
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
from tensorflow.keras.applications import InceptionV3, VGG19, ResNet101

In [ ]:
resnet = ResNet101()

In [ ]:
image_data_gen = ImageDataGenerator(rescale=1/255., horizontal_flip = True)

In [ ]:
# making size of image to 224, 224 as we are using ResNet101 and it requires to have 224,224 images when include_top is true
# which is by default true in resnet architecture.

X_train = image_data_gen.flow_from_directory(train_dir, target_size=(227,227), batch_size=32, class_mode="sparse")

X_val = image_data_gen.flow_from_directory(valid_dir, target_size=(227,227), batch_size=32, class_mode="sparse")

X_test = image_data_gen.flow_from_directory(test_dir, target_size=(227,227), batch_size=32, class_mode="sparse")

In [ ]:
resnet.summary()

In [ ]:
resnet.output

In [ ]:
x = resnet.output
# making output layer as a hidden layer and then adding a new hidden layer
x = Dense(1024, activation = "relu")(x)
# Add output layer
output = Dense(10, activation = "softmax")(x)

In [ ]:
model = Model(inputs = resnet.input, outputs = output)

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [ ]:
from tensorflow.python.ops import histogram_ops
from datetime import datetime as dt
def train_model():
  model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

  logdir = os.path.join("logs", dt.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
  model.fit(X_train, validation_data=X_val, epochs=50, callbacks=[tensorboard_callback])

In [ ]:
train_model()

In [ ]:
%tensorboard --logdir logs

In [ ]:
model.evaluate(X_test)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
y_pred[0]

In [ ]:
y_pred = np.argmax(y_pred, axis = 1)

In [ ]:
y_pred[:5]

In [ ]:
y_test = []
for i in range(int(X_test.n/32)+1):
  img, labels = next(X_test)
  for label in labels:
    y_test.append(label)

In [ ]:
y_test[:5]

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns

In [ ]:
cm = confusion_matrix(y_pred, y_test)

In [ ]:
cm

In [ ]:
sns.heatmap(cm)
plt.show()